# Исследование надежности заемщиков

**Описание проекта:**  
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.  
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

**Описание данных:**  
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

**Цель исследования:** 
Определить факторы, влияющие на возврат кредита в срок, для построения модели кредитного скоринга.

**Ход исследования:**
1) Загрузить и изучить данные из файла ```/datasets/data.csv```;
2) Предобработать данные: устранить пропуски, аномалии, дубликаты;
3) Провести исследовательский анализ:
    - Есть ли зависимость между количеством детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
4) Составить общий вывод

## Обзор данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можно медианным значением по каждому типу из столбца income_type:

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа в днях:

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) аномально большие значения стажа. Поскольку исправить такие значения не представляется возможным, оставим их как есть.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Есть два аномальных значения в столбце children -1 и 20, удалим их

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

В столбце days_employed заполним пропуски медианными значениями для каждого типа занятости income_type:

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [13]:
data['total_income'] = data['total_income'].astype(int)

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [13]:
data = data.drop_duplicates()

## Категоризация данных

Разделим клиентов на категории согласно их доходу

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Также разделим клиентов на категории по целям получения кредита:

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ

### Зависимость между количеством детей и возвратом кредита в срок

In [22]:
data.groupby('children')['debt'].agg(['mean', 'count', 'sum']) 

,mean,count,sum
children,,,
0,0.075438,14091,1063
1,0.092346,4808,444
2,0.094542,2052,194
3,0.081818,330,27
4,0.097561,41,4
5,0.000000,9,0


Клиентов с тремя и более детьми намного меньше, поэтому для большей наглядности клиентов можно разделить на категории.

In [23]:
def categorize_children(children):
    try:
        if children == 0:
            return 'нет детей'
        elif children == 1:
            return 'один ребенок'
        elif children == 2:
            return 'два ребенка'
        elif children > 2:    
            return 'многодетные'
    except:
        pass

In [24]:
data['children_category'] = data['children'].apply(categorize_children)

In [25]:
data.groupby('children_category')['debt'].agg(['mean', 'count', 'sum']).sort_values('count', ascending=False)

,mean,count,sum
children_category,,,
нет детей,0.075438,14091,1063
один ребенок,0.092346,4808,444
два ребенка,0.094542,2052,194
многодетные,0.081579,380,31


**Вывод:** Меньше задолженностей по возврату кредита имеют клиенты, у которых нет детей, доля должников среди таких клиентов составляет 7,5%. Хуже обстоит дело с клиентами, имеющими одного или двух детей, доля должников составляет 9,2% и 9,5%, соответственно. Клиенты, имеющие больше двух детей, представляют слишком маленькую выборку для корректной оценки.

### Зависимость между семейным положением и возвратом кредита в срок

In [26]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [27]:
data['family_status'] = data['family_status'].str.lower()

In [28]:
data.groupby('family_status')['debt'].agg(['mean', 'count', 'sum']).sort_values('mean') # Ваш код будет здесь. Вы можете создавать новые ячейки.

,mean,count,sum
family_status,,,
вдовец / вдова,0.066246,951,63
в разводе,0.070648,1189,84
женат / замужем,0.075606,12261,927
гражданский брак,0.093130,4134,385
не женат / не замужем,0.097639,2796,273


Объединим первые две группы

In [29]:
def categorize_family(family_status):
    try:
        if family_status == 'вдовец / вдова' or family_status == 'в разводе':
            return 'ранее состоявшие в браке'
        else:
            return family_status
    except:
        pass

In [30]:
data['family_category'] = data['family_status'].apply(categorize_family)

In [31]:
data['family_category'].value_counts() #проверка работы функции

женат / замужем             12261
гражданский брак             4134
не женат / не замужем        2796
ранее состоявшие в браке     2140
Name: family_category, dtype: int64

In [32]:
data.groupby('family_category')['debt'].agg(['mean', 'count', 'sum']).sort_values('mean')

,mean,count,sum
family_category,,,
ранее состоявшие в браке,0.068692,2140,147
женат / замужем,0.075606,12261,927
гражданский брак,0.093130,4134,385
не женат / не замужем,0.097639,2796,273


**Вывод:** Чаще остальных возвращают кредит в срок клиенты, ранее состоявшие в браке, доля должников среди них - 6,9%.
<br/> Среди клиентов, которые состоят в браке, 7,6% имеют задолженность по кредиту. Еще больший процент должников приходится на клиентов, находящихся в гражданском браке и в статусе "не женат/не замужем", 9,3% и 9,8%, соответственно. 

### Зависимость между уровнем дохода и возвратом кредита в срок

In [33]:
data['total_income_category'] = data['total_income_category'].\
    replace({'A':'A: выше 1000000','B': 'B: 200001-1000000', 'C': 'C: 50001–200000',\
             'D': 'D: 30001–50000', 'E': 'E: до 30000'})

In [34]:
data.groupby(by='total_income_category')['debt'].agg(['mean', 'count', 'sum']) # Ваш код будет здесь. Вы можете создавать новые ячейки.

,mean,count,sum
total_income_category,,,
A: выше 1000000,0.080000,25,2
B: 200001-1000000,0.070602,5014,354
C: 50001–200000,0.084982,15921,1353
D: 30001–50000,0.060172,349,21
E: до 30000,0.090909,22,2


**Вывод:** Клиенты со средним уровнем дохода от 50 000 до 200 000 наиболее часто берут кредит и имеют задолженность по его возврату, среди них 8,5% имеют долг. Напротив, клиенты с более высоким уровнем дохода, от 200 000 до 1 000 000, реже становятся должниками, таких - 7,1%.<br/>
Остальные группы клиентов малочисленны, поэтому сравнивать их нельзя.

### Как разные цели кредита влияют на его возврат в срок

In [35]:
data.groupby('purpose_category')['debt'].agg(['mean', 'count', 'sum']).sort_values('mean') 

,mean,count,sum
purpose_category,,,
операции с недвижимостью,0.072551,10751,780
проведение свадьбы,0.079118,2313,183
получение образования,0.092528,3988,369
операции с автомобилем,0.093480,4279,400


**Вывод:** Клиенты, у которых цели кредита связаны с недвижимостью, чаще погашают кредит в срок, доля должников среди таких клиентов составляет 7,3%. Возможно это связано с тем, что критерии для одобрения ипотеки более жесткие, чем для получения обычного кредита, поэтому ее получают потенциально более ответственные заемщики.

Процент клиентов, имеющих задолженность по возрату кредита на проведение свадьбы, составляет 7,9%. Еще более часто имеют долг клиенты, которые берут кредит на получение образования и операции с автомобилем, - 9,3% для каждой группы.

## Общий вывод

<div style="border:solid green 1px; padding: 10px"> 

По результатам исследования была выявлена зависимость между количеством детей и возвратом кредита в срок: доля должников меньше среди клиентов, не имеющих детей, 7,5%, чем среди клиентов, имеющих одного или двоих детей, соответственно 9,2% и 9,5%. Многодетных клиентов, имеющих больше двух детей, слишком мало для сравнения.
    
Также на возврат кредита влияет семейное положение. Наиболее часто возвращают кредит в срок клиенты, ранее состоявшие в браке, среди них на долю должников приходится 6,9%. Наибольшую группу составляют клиенты, находящиеся в браке, среди них доля должников - 7,6%. Еще менее надежными заемщиками являются клиенты, находящиеся в гражданском браке и холостые, - 9,3% и 9,8%, соответственно.
   
Влияние уровня дохода на возврат кредита в срок, можно оценить только для двух категорий клиентов: из наиболее обширной с доходом от 50000 до 200000 имеют долг по кредиту 8,5% и с доходом от 200000 до 1000000 - 7,1%. Остальные категории представляют слишком маленькие выборки для достоверной оценки.
    
Было оценено влияние цели получения кредита на его возврат в срок. Установлено, что наиболее надежными заемщиками являются клиенты, получающие кредит на недвижимость, таких клиентов большинство и среди них имеют долг 7,3%. Хуже возвращают кредиты на проведение свадьбы - 7,9% должников. Кредиты на образование и автомобили не выплачивают в срок 9,3% клиентов.

Таким образом, все вышеперечисленные факторы необходимо учитывать при построении модели кредитного скоринга. 
